In [1]:
import threading
import time
lock = threading.Lock()

### lock 与线程同步

```
lock.acquire()
# 临界区（critical section）。
# 读写一个共享资源
lock.release()
```

In [2]:
# 共享资源
shared_counter = 0

def increment_counter(n):
    global shared_counter
    for _ in range(n):
        lock.acquire()
        try:
            # 获取当前线程信息
            current_thread = threading.current_thread()
            print(f"Thread {current_thread.name} is incrementing the counter.")
            shared_counter += 1
        finally:
            lock.release()
        # 模拟一些处理时间
        time.sleep(0.1)

In [3]:
# 创建两个线程
thread1 = threading.Thread(target=increment_counter, args=(5, ), name='Thread-1')
thread2 = threading.Thread(target=increment_counter, args=(3, ), name='Thread-2')

# 启动线程
thread1.start()
thread2.start()

# 等待线程完成
thread1.join()
thread2.join()

Thread Thread-1 is incrementing the counter.
Thread Thread-2 is incrementing the counter.
Thread Thread-1 is incrementing the counter.
Thread Thread-2 is incrementing the counter.
Thread Thread-1 is incrementing the counter.
Thread Thread-2 is incrementing the counter.
Thread Thread-1 is incrementing the counter.
Thread Thread-1 is incrementing the counter.


In [4]:
shared_counter

8

### 线程间及进程间通信

1. 多线程通信
- 共享内存：线程**共享同一进程的内存空间**，可以直接访问全局变量或对象。
- 同步机制：由于竞争条件，需要使用锁（Lock）、条件变量（Condition）等同步原语。

2. 多进程通信：进程间通信（IPC）：**进程有独立的内存空间**，不能直接共享数据。
   
    通信方式：
    
    - 队列（Queue）：multiprocessing.Queue，用于在进程间传递数据。
    - 管道（Pipe）：multiprocessing.Pipe，创建双向或单向通信管道。
    - 共享内存：multiprocessing.Value和multiprocessing.Array，共享简单的数据类型和数组。
    - Manager对象：multiprocessing.Manager，提供共享的字典、列表等复杂数据类型。

#### 多线程

In [5]:
import threading

counter = 0
lock = threading.Lock()

def worker():
    global counter
    for _ in range(100000):
        with lock:
            counter += 1

threads = []
for _ in range(2):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("最终计数器值：", counter)

最终计数器值： 200000


- 共享变量：counter在所有线程间共享。
- 同步机制：使用Lock确保对counter的操作是原子性的，防止竞争条件。

#### 多进程

In [11]:
from multiprocessing import Process, Queue
import time

def worker(name, q):
    for i in range(5):
        message = f"子进程 {name}: 计算 {i} 的平方是 {i * i}, {q.qsize()}"
        q.put(message)
        time.sleep(0.1)  # 模拟耗时操作


q = Queue()
processes = []

# 创建并启动多个子进程
for i in range(3):
    p = Process(target=worker, args=(i, q))
    processes.append(p)
    p.start()

# 等待所有子进程完成
for p in processes:
    p.join()

In [12]:

# 从队列中获取子进程的结果
while not q.empty():
    print(q.get())

子进程 0: 计算 0 的平方是 0, 0
子进程 1: 计算 0 的平方是 0, 1
子进程 2: 计算 0 的平方是 0, 2
子进程 0: 计算 1 的平方是 1, 3
子进程 1: 计算 1 的平方是 1, 4
子进程 2: 计算 1 的平方是 1, 5
子进程 0: 计算 2 的平方是 4, 6
子进程 1: 计算 2 的平方是 4, 7
子进程 2: 计算 2 的平方是 4, 8
子进程 0: 计算 3 的平方是 9, 9
子进程 1: 计算 3 的平方是 9, 10
子进程 2: 计算 3 的平方是 9, 11
子进程 0: 计算 4 的平方是 16, 12
子进程 1: 计算 4 的平方是 16, 13
子进程 2: 计算 4 的平方是 16, 14


#### 管道与双向通信

- 使用 `Pipe()` 函数创建一个管道，管道两端的连接对象：
    - Pipe() 函数返回一对连接对象，通常命名为 conn1 和 conn2，它们代表管道的两端。
    - 这两个连接对象可以在不同的进程中使用，实现双向通信。
- 通信方向：
    - 当在一个连接对象上调用 send() 方法时，消息会被发送到另一端的连接对象上。
    - 换句话说，conn1.send(msg) 发送的消息需要在 conn2.recv() 中接收，反之亦然。
- p2p 基础通信源语
    - send & recv

In [16]:
from multiprocessing import Process, Pipe
import time

def worker(child_conn):
    # 在子进程中，接收来自父进程的消息
    msg_from_parent = child_conn.recv()
    print(f"[子进程] 收到来自父进程的消息：{msg_from_parent}")

    # 处理数据
    response = msg_from_parent.upper()
    time.sleep(1)  # 模拟耗时操作

    # 在子进程中，发送消息给父进程
    child_conn.send(f"{response} - 来自子进程")
    child_conn.close()

In [18]:
# 创建一个双向管道，获得父进程和子进程各自的连接对象
parent_conn, child_conn = Pipe()

# 创建并启动子进程，将子进程的连接对象传递给它
p = Process(target=worker, args=(child_conn,))
p.start()

# 在父进程中，向子进程发送消息
message = "hello from parent"
print(f"[父进程] 发送消息给子进程：{message}")
parent_conn.send(message)

# 在父进程中，接收来自子进程的响应
response_from_child = parent_conn.recv()
print(f"[父进程] 收到来自子进程的响应：{response_from_child}")

p.join()


[子进程] 收到来自父进程的消息：hello from parent
[父进程] 发送消息给子进程：hello from parent
[父进程] 收到来自子进程的响应：HELLO FROM PARENT - 来自子进程
